# Abstratct
- 

# Introduction
- For the creation of a better world, good mental health is required. Medical science progressed outstandingly, but still, some diseases exist which can not be identified earlier to curve them. Natural language processing (NLP) and deep learning (DL) combination can be used to identify such diseases by the classification algorithm.

- Alzheimer's is one of the diseases which hard to detect in earlier phases or even in the present moment. Fortunately, Data science has the solution to it. A combination of natural language processing and deep learning can be used to detect the disease. Sentences can be transformed to word embedding by NLP. where the representation of words into vectors of real numbers. This can be done by BERT (Bidirectional Encoder Representations from Transformers). Berta has different variation such as ROBERTA, ELECTRA.

- ROBERTA (Robustly Optimized BERT-Pretraining Approach) is powerful model for word embedding. It transform each sentences into token and these token is used by model for word embedding in encoder phase. 

- Gaussian process classification is useful for this high dimensional word embedding, as It has a huge uncertainty in data.

In [1]:
# environment for this notbook
import torch

from transformers import RobertaModel, RobertaModel, RobertaTokenizer

import pandas as pd

import glob

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

from scipy.stats import entropy


### Dataset

- There is two type of datasets. One with sentences and second with the patient health discription. First  type of data is used for this word embedding task. Data collected by image desciption of patients affected by Alzeihmer and helathy person.

### Pre-processing

- There is couple of csv file based on patients description and therefore It is important to convert them into appropriate formate for word embedding. Through combination of all csv file, make it single one and add label to identify which is Alzeihmer disease (AD) and healthy control(HC). Similarly, same process performed for test data set.

In [2]:
# Combine all csv files of AD_train and add label column
AD_train=[]
path=r"C:\Users\uttam\OneDrive\Desktop\NLP Project\NLP-Project\Data\AD_train"
csv_files=glob.glob(path+'/*.csv')
for f in csv_files:
    df1 = pd.read_csv(f)
    df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]
    AD_train.append(df1)


df_AD_train=pd.concat(AD_train,axis=0,ignore_index=True)
label_AD=0 #AD : 0 label
df_AD_train['label']=label_AD

In [3]:
# Combine all csv files of AD_test and add label column
AD_test=[]
path=r"C:\Users\uttam\OneDrive\Desktop\NLP Project\NLP-Project\Data\AD_train\AD_Test"
csv_files=glob.glob(path+'/*.csv')
for f in csv_files:
    df1 = pd.read_csv(f)
    df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]
    AD_test.append(df1)


df_AD_test=pd.concat(AD_test,axis=0,ignore_index=True)
label_AD=0 #AD : 0 label
df_AD_test['label']=label_AD

In [4]:
# Combine all csv files of HC_train and add label column
HC_train=[]
path=r"C:\Users\uttam\OneDrive\Desktop\NLP Project\NLP-Project\Data\HC_train"
csv_files=glob.glob(path+'/*.csv')
for f in csv_files:
    df2=pd.read_csv(f)
    df2 = df2.loc[:, ~df2.columns.str.contains('^Unnamed')]
    HC_train.append(df2)


df_HC_train=pd.concat(HC_train,axis=0,ignore_index=True)
label_HC=1 #HC : 1 label
df_HC_train['label']=label_HC


In [5]:
# Combine all csv files of HC_test and add label column
HC_test=[]
path=r"C:\Users\uttam\OneDrive\Desktop\NLP Project\NLP-Project\Data\HC_train\HC_Test"
csv_files=glob.glob(path+'/*.csv')
for f in csv_files:
    df2=pd.read_csv(f)
    df2 = df2.loc[:, ~df2.columns.str.contains('^Unnamed')]
    HC_test.append(df2)


df_HC_test=pd.concat(HC_test,axis=0,ignore_index=True)
label_HC=1 #HC : 1 label
df_HC_test['label']=label_HC

In [6]:
# concatination of AD & HC train and similarly for test set
df_train=pd.concat([df_AD_train,df_HC_train], ignore_index=True)
df_test=pd.concat([df_AD_test,df_HC_test], ignore_index=True)

- Furthermore, It is essential to shuffle data samples to avoid  overfitting and also help to create robust classification model. After pre-processing step, There is total 1968 samples in train set and 649 samples in test set. It is important to separate train and test set in starting to avoid mixture of sentences from one scipt to another one, because it is essential to maintain uniqness of patients' script.



In [7]:
# shuffling
df_train = df_train.sample(frac=1,random_state=0,replace=True).reset_index(drop=True)
df_test = df_test.sample(frac=1,random_state=0,replace=True).reset_index(drop=True)
print('length of train', len(df_train),'.','length of test', len(df_test)) # output of train and test length
df_train

length of train 1968 . length of test 649


,sentence,label
0,I heard it might but I'm sayin(g) let's tell h...,0
1,yes .,0
2,and the woman is standing in water .,1
3,the little girl's got her hand up for one .,1
4,wonder if he got a cookie .,0
...,...,...
1963,fell down .,0
1964,uhhuh .,1
1965,that's a mess .,0
1966,touching lip .,1


### Word Embedding

- The Next step is word embedding of sentences. For the word embedding, Tokenizer and Roberta model required. Tokenizer is helpful to tokenize words of sentenc. Roberta model use 'G' character to represt space as described in given below example. 

- Example: 'My name is Khodiyar'
           ['My','Gname', 'Gis', 'GKhodiyar'] => Tokenized sentence

- Transformer convert each tokenized word into unique id which can be taken by roberta model for further process. Here, tokenizer add two addition tokens in starting and ending of each sentence. For the distinguish each sentence to each other.

- For more convenience in this task we make same length for all sentences by taking into account length of longest sentence wich is 47 in our case. Extra added tokens are zero which will not affect to the model. Roberta model config can be checked by **model.config**. 

In [8]:
# loading of tokenizer and roberta model
tokenizer=RobertaTokenizer.from_pretrained('roberta-base')
Roberta = RobertaModel.from_pretrained('roberta-base',output_hidden_states=True)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
sentences_train=df_train['sentence'] # train
sentences_test=df_test['sentence']  # test

In [10]:
sentences_pool={'sentences_train':sentences_train,'sentences_test':sentences_test} # train and test pool
input_id_train=[] # list
input_id_test=[]  # list
max_len = 47

# convert sentence into token
for key in sentences_pool.keys():
    input_ids = [torch.tensor(tokenizer.encode(sentence, add_special_tokens=True)) for sentence in sentences_pool[key]]

    # Pad the sequences to have the same length
    
    padded_input = [torch.cat((input_id, torch.zeros((max_len - len(input_id)), dtype=torch.long))) for input_id in input_ids]
    padded_input = torch.stack(padded_input).to(torch.int64) # tensor

    if key=='sentences_train':
        input_id_train.append(padded_input)
    else :
        input_id_test.append(padded_input)
    

- After transforming words into id, roberta model used. Word embedding can be achieved by keeping gradient off. This means that PyTorch will not keep track of the operations that are involved in computing the word embeddings, and therefore it will not use them to compute the gradients during backpropagation. By using torch.no_grad() when computing the word embeddings, we can reduce the memory usage and computation time required during forward propagation, as well as avoid any potential errors that may occur if we accidentally try to compute gradients for the word embeddings. Additionally, disabling gradient tracking can also help to improve the speed and efficiency of the overall training process.

In [11]:
input_id_train=input_id_train[0] # get tensor from list

#  word embedding of train
with torch.no_grad():
    output = Roberta(input_id_train)
    last_hidden_state_train = output[2] # Word embedding can get by input for next part like forward pass

In [12]:
input_id_test=input_id_test[0] # get tensor from list

# word embedding of test
with torch.no_grad():
    output = Roberta(input_id_test)
    last_hidden_state_test = output[2]

- Roberta model has 12 encoder and each encoder create one embedding, It gives total twelve embeddings. Every token has 769 length. Best encodding can be achieved by combination of encoders, in this case sum of last 7 encoders' embedding performed better for classification task. Now, each sentence has dimension of **[1,47,768]**. Entropy can help to reduce dimension. Entropy convert array into single value

In [13]:
# find best embedding by combination of encoders and use entropy to convert sentence dimension from [1,47,768] to [1,768]

'''
Dimension: shape of whole dataset after embedding: [2617,47,768],
           each sentence dimension: [1,47,768] shape 
           each word dimension: [1,768]
'''
x_train=[]
x_test=[]
embedding_pool={'x_train':last_hidden_state_train, 'x_test':last_hidden_state_test}

for key in embedding_pool.keys():
    final_embedding=torch.stack(embedding_pool[key][-7:]).sum(0) # combine output of last six encoder
    print('Min value of', key,'is:', torch.min(final_embedding))
    final_embedding=final_embedding+torch.max(final_embedding) # deviation from negative to positive for entropy
    print('Total dimension of', key, 'is (without entorpy):' ,final_embedding.shape)
    # normalized=normalize(x, p=1, dim=0)
    # print(normalized.shape)
    final_embedding = entropy(final_embedding, axis=1) 
    print('Final embedding shape', key, 'is (after entropy):',final_embedding.shape) # one dimensional single sentence of length 768
    print('----------')
    if key=='x_train':
        x_train.append(final_embedding)
    else:
        x_test.append(final_embedding)


Min value of x_train is: tensor(-84.2695)
Total dimension of x_train is (without entorpy): torch.Size([1968, 47, 768])
Final embedding shape x_train is (after entropy): (1968, 768)
----------
Min value of x_test is: tensor(-81.4422)
Total dimension of x_test is (without entorpy): torch.Size([649, 47, 768])
Final embedding shape x_test is (after entropy): (649, 768)
----------


In [1]:
final_embedding

NameError: name 'final_embedding' is not defined

## Classification

In [14]:
# classification task
x_train, x_test, y_train, y_test= x_train[0], x_test[0], df_train['label'], df_test['label']

# Classification model: GaussianProcessClassifier 
Gaussian = GaussianProcessClassifier(kernel=1.0 * RBF(1.0),max_iter_predict=1000)


In [15]:
Gaussian.fit(x_train,y_train) # fit to the model
y_pred=Gaussian.predict(x_test) # evaluation

In [16]:
# accuracy score (result vary as dataset shuffle each time:54%-68%)
print(f"print score for :{metrics.accuracy_score(y_test,y_pred)}")

print score for :0.6024653312788906


In [17]:
# classification report
class_labels=  [0,1] # AD : 0, HC : 1
print(classification_report(y_test, y_pred, labels=class_labels)) 

              precision    recall  f1-score   support

           0       0.65      0.60      0.62       355
           1       0.56      0.60      0.58       294

    accuracy                           0.60       649
   macro avg       0.60      0.60      0.60       649
weighted avg       0.61      0.60      0.60       649

